In [1]:
import pandas as pd
import numpy as np
import json
import time
import re
import requests
from bs4 import BeautifulSoup
import itertools
import datetime

In [20]:
pd.set_option('display.max_columns', None)
pd.options.display.max_colwidth = 100

# Gather list of past events

In [ ]:
completed = 'http://ufcstats.com/statistics/events/completed?page=all'
url_request = requests.get(completed).text
soup = BeautifulSoup(url_request, 'html.parser')

In [ ]:
events_data = []
links_data = []
table = soup.find('table',{'class':'b-statistics__table-events'})
table_rows = table.find_all('tr')
for row in table_rows[3:]:
    events_data.append([t.text.strip() for t in row.find_all('td')]) 
    links_data.append(row.find('a')['href'])

In [ ]:
events_df = pd.DataFrame(events_data, columns=['Name-Date', 'Location'])
events_df['url'] = links_data
events_df

In [ ]:
events_df['Event'] = events_df['Name-Date'].str.split("\n",expand=True)[0].str.strip()
events_df['Date'] = events_df['Name-Date'].str.split("\n",expand=True)[3].str.strip()

In [ ]:
events_df = events_df[['Date', 'Event', 'Location', 'url']]

In [ ]:
events_df.to_csv("data_ufcstats/ufc-stats-events-list.csv", index=False)

# Gather event detail from each event

In [ ]:
events_df = pd.read_csv("data_ufcstats/ufc-stats-events-list.csv")
events_df.head()

In [ ]:
# Get event details from each event
matches_df = pd.DataFrame()
for index, row in events_df.iterrows():
    event_name = row['Event']
    event_url = row['url']
    event_date = row['Date']
    print(str(index) + "\t" + event_date + "\t" + event_name)
    print(str(event_url))
    try:
        url_request = requests.get(event_url).text
        soup = BeautifulSoup(url_request, 'html.parser')
        table_rows = soup.find_all('tr',{'class':'b-fight-details__table-row'})
        data = []
        for row in table_rows[1:]:
            data.append([t.text.strip() for t in row.find_all('td')]) 
        df = pd.DataFrame(data, columns=['Results', 'Fighters', 'STR', 'TD', 'SUB', 'PASS', 'WEIGHT CLASS', 'METHOD', 'ROUND', 'TIME'])
        df.insert(0, "Date", event_date) 
        df.insert(1, "Event", event_name)
    except:
        print("Error")
    matches_df = matches_df.append(df, ignore_index=True)


In [ ]:
matches_df = matches_df[matches_df['Results'].notna()]
matches_df = matches_df.drop_duplicates()
matches_df['Date']= pd.to_datetime(matches_df['Date'].str.strip(),format= '%B %d, %Y')

In [ ]:
print("Total number of matches: " + str(len(matches_df)))
print("Total number of events: " +  str(len(matches_df['Event'].unique())))
print("Total number of dates: " +  str(len(matches_df['Date'].unique())))

In [ ]:
matches_df['Results_1'] = matches_df['Results'].str.split("\n",expand=True)[0]
matches_df['Results_2'] = matches_df['Results'].str.split("\n",expand=True)[3]
matches_df['Fighter_1'] = matches_df['Fighters'].str.split("\n",expand=True)[0]
matches_df['Fighter_2'] = matches_df['Fighters'].str.split("\n",expand=True)[5]
matches_df['STR_1'] = matches_df['STR'].str.split("\n",expand=True)[0]
matches_df['STR_2'] = matches_df['STR'].str.split("\n",expand=True)[6]
matches_df['TD_1'] = matches_df['TD'].str.split("\n",expand=True)[0]
matches_df['TD_2'] = matches_df['TD'].str.split("\n",expand=True)[4]
matches_df['SUB_1'] = matches_df['SUB'].str.split("\n",expand=True)[0]
matches_df['SUB_2'] = matches_df['SUB'].str.split("\n",expand=True)[4]
matches_df['PASS_1'] = matches_df['PASS'].str.split("\n",expand=True)[0]
matches_df['PASS_2'] = matches_df['PASS'].str.split("\n",expand=True)[4]
matches_df['METHOD_result'] = matches_df['METHOD'].str.split("\n",expand=True)[0]
matches_df['METHOD_note'] = matches_df['METHOD'].str.split("\n",expand=True)[4]

matches_df = matches_df[['Date', 'Event', 'ROUND', 'TIME', 'Results_1', 'Results_2',
       'Fighter_1', 'Fighter_2', 'WEIGHT CLASS', 'STR_1', 'STR_2', 'TD_1', 'TD_2', 'SUB_1',
       'SUB_2', 'PASS_1', 'PASS_2', 'METHOD_result', 'METHOD_note']]

matches_df = matches_df.apply(lambda x: x.str.strip() if x.dtype == "object" else x)

In [ ]:
matches_df.to_csv("data_ufcstats/ufc-stats-matches-overview.csv", index=False)
matches_df.columns

# Collect links to detailed match data for each event

In [ ]:
matches_data_df = pd.DataFrame()
for index, row in events_df.iterrows():
    event_name = row['Event']
    event_url = row['url']
    event_date = row['Date']
#     print(str(index) + "\t" + event_date + "\t" + event_name)
#     print(str(event_url))
    try:
        url_request = requests.get(event_url).text
        soup = BeautifulSoup(url_request, 'html.parser')
        table_rows = soup.find_all('tr',{'class':'js-fight-details-click'})
        data = []
        for row in table_rows:
            data.append(row['data-link'])
        df = pd.DataFrame(data, columns=['match_url'])
        df.insert(0, "Date", event_date) 
        df.insert(1, "Event", event_name)
    except:
        print("Error")
    matches_data_df = matches_data_df.append(df, ignore_index=True)
matches_data_df['Date']= pd.to_datetime(matches_df['Date'].str.strip(),format= '%B %d, %Y')

In [ ]:
matches_data_df.to_csv("data_ufcstats/ufc-stats-matches-links.csv", index=False)

# Gather detailed match data 

In [8]:
matches_data_df = pd.read_csv("data_ufcstats/ufc-stats-matches-links.csv")
matches_data_df.tail()

,Date,Event,match_url
5602,2020-05-30,UFC Fight Night: Woodley vs. Burns,http://ufcstats.com/fight-details/1e9a2b197ebf...
5603,2020-05-30,UFC Fight Night: Woodley vs. Burns,http://ufcstats.com/fight-details/a6743d41b0b9...
5604,2020-05-30,UFC Fight Night: Woodley vs. Burns,http://ufcstats.com/fight-details/57dec2b07b75...
5605,2020-05-30,UFC Fight Night: Woodley vs. Burns,http://ufcstats.com/fight-details/3640defb8294...
5606,2020-05-30,UFC Fight Night: Woodley vs. Burns,http://ufcstats.com/fight-details/2300ad7f8708...


In [9]:
# Get event details from each event
full_matches_df = pd.DataFrame()
for index, row in matches_data_df.iterrows():
    event_name = row['Event']
    match_url = row['match_url']
    event_date = row['Date']
    print(str(index) + "\t" + event_date + "\t" + event_name)
    print(str(match_url))
    try:
        url_request = requests.get(match_url).text
        soup = BeautifulSoup(url_request, 'html.parser')
        
        fight_details = soup.find('div',{'class':'b-fight-details__fight'})
        description = [t.text.strip() for t in fight_details.find_all('p')][0]
        description = ' '.join(description.split())
        item_df = pd.DataFrame([description], columns=['description'])

        p = re.compile(r'\S+: ')
        item_df['METHOD'] = p.split(description)[1]
        item_df['ROUND'] = p.split(description)[2].strip()
        item_df['TIME'] = p.split(description)[3].split(" ")[0]
        item_df['TIME_FORMAT'] = p.split(description)[4].strip()
        if len(p.split(description)) < 6:
            item_df['REFEREE'] = ""
        else:
            item_df['REFEREE'] = p.split(description)[5]
        item_df = item_df.drop(columns=['description'])

        item_df.insert(0, "Date", event_date) 
        item_df.insert(1, "Event", event_name)
        
        table_rows = soup.find_all('table',{'cellpadding':None})

        lines = []
        for row in table_rows[0::2]:
            lines.append([t.text.strip() for t in row.find_all('td')])

        # FIGHTER	KD	SIG. STR.	SIG. STR. %	TOTAL STR.	TD	TD %	SUB. ATT	PASS	REV.
        df = pd.DataFrame(lines[0]).transpose()

        item_df['Fighter_1'] = df[0].str.split("\n",expand=True)[0]
        item_df['Fighter_2'] = df[0].str.split("\n",expand=True)[3]
        item_df['KD_1'] = df[1].str.split("\n",expand=True)[0]
        item_df['KD_2'] = df[1].str.split("\n",expand=True)[3]
        item_df['SIG_STR_1'] = df[2].str.split("\n",expand=True)[0]
        item_df['SIG_STR_2'] = df[2].str.split("\n",expand=True)[3]
        item_df['pSIG_STR_1'] = df[3].str.split("\n",expand=True)[0]
        item_df['pSIG_STR_2'] = df[3].str.split("\n",expand=True)[3]
        item_df['TOTAL_STR_1'] = df[4].str.split("\n",expand=True)[0]
        item_df['TOTAL_STR_2'] = df[4].str.split("\n",expand=True)[3]
        item_df['TD_1'] = df[5].str.split("\n",expand=True)[0]
        item_df['TD_2'] = df[5].str.split("\n",expand=True)[3]
        item_df['pTD_1'] = df[6].str.split("\n",expand=True)[0]
        item_df['pTD_2'] = df[6].str.split("\n",expand=True)[3]
        item_df['SUB_ATT_1'] = df[7].str.split("\n",expand=True)[0]
        item_df['SUB_ATT_2'] = df[7].str.split("\n",expand=True)[3]
        item_df['PASS_1'] = df[8].str.split("\n",expand=True)[0]
        item_df['PASS_2'] = df[8].str.split("\n",expand=True)[3]
        item_df['REV_1'] = df[9].str.split("\n",expand=True)[0]
        item_df['REV_2'] = df[9].str.split("\n",expand=True)[3]

        # FIGHTER	SIG. STR	SIG. STR. %	HEAD	BODY	LEG	DISTANCE	CLINCH	GROUND
        df = pd.DataFrame(lines[1]).transpose()

        item_df['HEAD_1'] = df[3].str.split("\n",expand=True)[0]
        item_df['HEAD_2'] = df[3].str.split("\n",expand=True)[3]
        item_df['BODY_1'] = df[4].str.split("\n",expand=True)[0]
        item_df['BODY_2'] = df[4].str.split("\n",expand=True)[3]
        item_df['LEG_1'] = df[5].str.split("\n",expand=True)[0]
        item_df['LEG_2'] = df[5].str.split("\n",expand=True)[3]
        item_df['DISTANCE_1'] = df[6].str.split("\n",expand=True)[0]
        item_df['DISTANCE_2'] = df[6].str.split("\n",expand=True)[3]
        item_df['CLINCH_1'] = df[7].str.split("\n",expand=True)[0]
        item_df['CLINCH_2'] = df[7].str.split("\n",expand=True)[3]
        item_df['GROUND_1'] = df[8].str.split("\n",expand=True)[0]
        item_df['GROUND_2'] = df[8].str.split("\n",expand=True)[3]
        
    except:
        print("Error")
        item_df['Fighter_1'] = match_url
        item_df['Fighter_2'] = match_url
    full_matches_df = full_matches_df.append(item_df, ignore_index=True)

0	1993-11-12	UFC 1: The Beginning
http://ufcstats.com/fight-details/567a09fd200cfa05
1	1993-11-12	UFC 1: The Beginning
http://ufcstats.com/fight-details/2d2bbc86e941e05c
2	1993-11-12	UFC 1: The Beginning
http://ufcstats.com/fight-details/cecdc0da584274b9
3	1993-11-12	UFC 1: The Beginning
http://ufcstats.com/fight-details/46acd54cc0c905fb
4	1993-11-12	UFC 1: The Beginning
http://ufcstats.com/fight-details/ac7ca2ec38b96c1a
5	1993-11-12	UFC 1: The Beginning
http://ufcstats.com/fight-details/ffd16691c4c4aafc
6	1993-11-12	UFC 1: The Beginning
http://ufcstats.com/fight-details/00b0796724ec1c09
7	1993-11-12	UFC 1: The Beginning
http://ufcstats.com/fight-details/64139d1d505e46c5
8	1994-03-11	UFC 2: No Way Out
http://ufcstats.com/fight-details/d917c8c7461b9831
9	1994-03-11	UFC 2: No Way Out
http://ufcstats.com/fight-details/3b020d4914b44fc8
10	1994-03-11	UFC 2: No Way Out
http://ufcstats.com/fight-details/ccee020be2e8d9f6
11	1994-03-11	UFC 2: No Way Out
http://ufcstats.com/fight-details/aa161c7

/Users/cindyhui/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:7123: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort,


35	1994-12-16	UFC 4: Revenge of the Warriors
http://ufcstats.com/fight-details/0b71ada6c9209fc9
36	1994-12-16	UFC 4: Revenge of the Warriors
http://ufcstats.com/fight-details/8f8f47a728c3127e
37	1994-12-16	UFC 4: Revenge of the Warriors
http://ufcstats.com/fight-details/a1db4c917777aa79
Error
38	1994-12-16	UFC 4: Revenge of the Warriors
http://ufcstats.com/fight-details/7e7d7a488ea21f63
39	1995-04-07	UFC 5: The Return of the Beast
http://ufcstats.com/fight-details/db8df615610f3632
40	1995-04-07	UFC 5: The Return of the Beast
http://ufcstats.com/fight-details/6ca94b35719eb300
41	1995-04-07	UFC 5: The Return of the Beast
http://ufcstats.com/fight-details/db4c169722cf1e33
42	1995-04-07	UFC 5: The Return of the Beast
http://ufcstats.com/fight-details/cbf3fc8730b4ad09
43	1995-04-07	UFC 5: The Return of the Beast
http://ufcstats.com/fight-details/ebfe035b70080668
44	1995-04-07	UFC 5: The Return of the Beast
http://ufcstats.com/fight-details/802c0e4e228996ac
45	1995-04-07	UFC 5: The Return of

123	1997-02-07	UFC 12: Judgement Day
http://ufcstats.com/fight-details/fa347a9ac9d0558c
124	1997-02-07	UFC 12: Judgement Day
http://ufcstats.com/fight-details/2963e9a4b8c10bc9
125	1997-02-07	UFC 12: Judgement Day
http://ufcstats.com/fight-details/b28809d58f8c18fd
126	1997-02-07	UFC 12: Judgement Day
http://ufcstats.com/fight-details/e45feb9866eb2da7
127	1997-02-07	UFC 12: Judgement Day
http://ufcstats.com/fight-details/18af785da73b5996
128	1997-02-07	UFC 12: Judgement Day
http://ufcstats.com/fight-details/77bf1e37929b0d59
Error
129	1997-02-07	UFC 12: Judgement Day
http://ufcstats.com/fight-details/565ecefd8a37ad7e
Error
130	1997-02-07	UFC 12: Judgement Day
http://ufcstats.com/fight-details/72e7d6d5328bc3d6
131	1997-05-30	UFC 13: The Ultimate Force
http://ufcstats.com/fight-details/0e16795e385e76a7
132	1997-05-30	UFC 13: The Ultimate Force
http://ufcstats.com/fight-details/4165dd0c39926e2f
133	1997-05-30	UFC 13: The Ultimate Force
http://ufcstats.com/fight-details/86f566f142593667
134	1

213	1999-07-16	UFC 21: Return of the Champions
http://ufcstats.com/fight-details/3e4199f9a69bac88
214	1999-07-16	UFC 21: Return of the Champions
http://ufcstats.com/fight-details/693e4a0bed9a1369
215	1999-07-16	UFC 21: Return of the Champions
http://ufcstats.com/fight-details/1db8bed8b4d30fb3
216	1999-07-16	UFC 21: Return of the Champions
http://ufcstats.com/fight-details/82cf32aea3d7edf3
217	1999-09-24	UFC 22: Only One Can be Champion
http://ufcstats.com/fight-details/b1134affa18449dd
218	1999-09-24	UFC 22: Only One Can be Champion
http://ufcstats.com/fight-details/96c247ba0eb1ac65
219	1999-09-24	UFC 22: Only One Can be Champion
http://ufcstats.com/fight-details/eebf672428b168b7
220	1999-09-24	UFC 22: Only One Can be Champion
http://ufcstats.com/fight-details/c6ead49500c047e5
221	1999-09-24	UFC 22: Only One Can be Champion
http://ufcstats.com/fight-details/d9e86958bcb15c26
222	1999-09-24	UFC 22: Only One Can be Champion
http://ufcstats.com/fight-details/a1c1e3c1e9c6cf1e
223	1999-09-24

...

5216	2019-07-13	UFC Fight Night: De Randamie vs. Ladd
http://ufcstats.com/fight-details/b436dda934059631
5217	2019-07-13	UFC Fight Night: De Randamie vs. Ladd
http://ufcstats.com/fight-details/60be02c5566c8348
5218	2019-07-13	UFC Fight Night: De Randamie vs. Ladd
http://ufcstats.com/fight-details/37a8c442fb38db75
5219	2019-07-13	UFC Fight Night: De Randamie vs. Ladd
http://ufcstats.com/fight-details/ad63c30e22e63344
5220	2019-07-13	UFC Fight Night: De Randamie vs. Ladd
http://ufcstats.com/fight-details/2d42ef44ad2470e1
5221	2019-07-13	UFC Fight Night: De Randamie vs. Ladd
http://ufcstats.com/fight-details/358468724d7d4322
5222	2019-07-13	UFC Fight Night: De Randamie vs. Ladd
http://ufcstats.com/fight-details/aa6a9aeeb45577d4
5223	2019-07-20	UFC Fight Night: Dos Anjos vs. Edwards
http://ufcstats.com/fight-details/9d6b46a3cf5ec494
5224	2019-07-20	UFC Fight Night: Dos Anjos vs. Edwards
http://ufcstats.com/fight-details/816662e622987a14
5225	2019-07-20	UFC Fight Night: Dos Anjos vs. Edward

5296	2019-09-07	UFC 242: Khabib vs. Poirier
http://ufcstats.com/fight-details/357a34207d70da69
5297	2019-09-07	UFC 242: Khabib vs. Poirier
http://ufcstats.com/fight-details/849eb47d349f3fe4
5298	2019-09-07	UFC 242: Khabib vs. Poirier
http://ufcstats.com/fight-details/066756ea40b134a7
5299	2019-09-07	UFC 242: Khabib vs. Poirier
http://ufcstats.com/fight-details/8accd04438b91262
5300	2019-09-07	UFC 242: Khabib vs. Poirier
http://ufcstats.com/fight-details/77f123712368bcbc
5301	2019-09-07	UFC 242: Khabib vs. Poirier
http://ufcstats.com/fight-details/bef4df43d6052a02
5302	2019-09-07	UFC 242: Khabib vs. Poirier
http://ufcstats.com/fight-details/8e1ae2ae6694b131
5303	2019-09-07	UFC 242: Khabib vs. Poirier
http://ufcstats.com/fight-details/2c7b788effdc1b4c
5304	2019-09-07	UFC 242: Khabib vs. Poirier
http://ufcstats.com/fight-details/18aff757c54687f1
5305	2019-09-07	UFC 242: Khabib vs. Poirier
http://ufcstats.com/fight-details/43496ff308e681a0
5306	2019-09-07	UFC 242: Khabib vs. Poirier
http:/

5376	2019-10-18	UFC Fight Night: Reyes vs. Weidman
http://ufcstats.com/fight-details/baf17e5e1fe1c744
5377	2019-10-18	UFC Fight Night: Reyes vs. Weidman
http://ufcstats.com/fight-details/9c3fb95a1558b38d
5378	2019-10-18	UFC Fight Night: Reyes vs. Weidman
http://ufcstats.com/fight-details/38d6c2538ce6ec4c
5379	2019-10-18	UFC Fight Night: Reyes vs. Weidman
http://ufcstats.com/fight-details/26d75187369ecf2c
5380	2019-10-18	UFC Fight Night: Reyes vs. Weidman
http://ufcstats.com/fight-details/74b44635535cb290
5381	2019-10-18	UFC Fight Night: Reyes vs. Weidman
http://ufcstats.com/fight-details/1e3d3545b6dd13a7
5382	2019-10-18	UFC Fight Night: Reyes vs. Weidman
http://ufcstats.com/fight-details/93f3e9b9f093ad57
5383	2019-10-26	UFC Fight Night: Maia vs. Askren
http://ufcstats.com/fight-details/241a0833e1c76804
5384	2019-10-26	UFC Fight Night: Maia vs. Askren
http://ufcstats.com/fight-details/3162ae3ccbf785ac
5385	2019-10-26	UFC Fight Night: Maia vs. Askren
http://ufcstats.com/fight-details/b2d

5458	2019-12-21	UFC Fight Night: Edgar vs. The Korean Zombie
http://ufcstats.com/fight-details/2b8cae68963a40a9
5459	2019-12-21	UFC Fight Night: Edgar vs. The Korean Zombie
http://ufcstats.com/fight-details/997ab1ea9fab5643
5460	2019-12-21	UFC Fight Night: Edgar vs. The Korean Zombie
http://ufcstats.com/fight-details/b3bd27affa038e9c
5461	2019-12-21	UFC Fight Night: Edgar vs. The Korean Zombie
http://ufcstats.com/fight-details/60e1b3ac3182f5b3
5462	2019-12-21	UFC Fight Night: Edgar vs. The Korean Zombie
http://ufcstats.com/fight-details/3637e661b6854b49
5463	2019-12-21	UFC Fight Night: Edgar vs. The Korean Zombie
http://ufcstats.com/fight-details/6ea23c2c30f85a89
5464	2019-12-21	UFC Fight Night: Edgar vs. The Korean Zombie
http://ufcstats.com/fight-details/9140e6fdeaa7cd3f
5465	2019-12-21	UFC Fight Night: Edgar vs. The Korean Zombie
http://ufcstats.com/fight-details/5a5a65d8df176506
5466	2019-12-21	UFC Fight Night: Edgar vs. The Korean Zombie
http://ufcstats.com/fight-details/75523d319

5538	2020-02-29	UFC Fight Night: Benavidez vs. Figueiredo
http://ufcstats.com/fight-details/533d0b5c0adfa15f
5539	2020-02-29	UFC Fight Night: Benavidez vs. Figueiredo
http://ufcstats.com/fight-details/d9a218ac3e148541
5540	2020-02-29	UFC Fight Night: Benavidez vs. Figueiredo
http://ufcstats.com/fight-details/e30b9829e42aa4ca
5541	2020-03-07	UFC 248: Adesanya vs. Romero
http://ufcstats.com/fight-details/891470d896b1de5a
5542	2020-03-07	UFC 248: Adesanya vs. Romero
http://ufcstats.com/fight-details/b46f2f007b622bce
5543	2020-03-07	UFC 248: Adesanya vs. Romero
http://ufcstats.com/fight-details/cf0a6466b762c668
5544	2020-03-07	UFC 248: Adesanya vs. Romero
http://ufcstats.com/fight-details/d66a8591af39ca7b
5545	2020-03-07	UFC 248: Adesanya vs. Romero
http://ufcstats.com/fight-details/a70c43cafabc73c3
5546	2020-03-07	UFC 248: Adesanya vs. Romero
http://ufcstats.com/fight-details/cd258394e8d58d04
5547	2020-03-07	UFC 248: Adesanya vs. Romero
http://ufcstats.com/fight-details/4d607b6ddc77a34c
5

In [22]:
# full_matches_df.loc[33]['Fighter_1'] = "Marcus Bossett"
# full_matches_df.loc[33]['Fighter_2'] = "Eldo Xavier Dias" 

# full_matches_df.loc[37]['Fighter_1'] = "Joe Charles"      
# full_matches_df.loc[37]['Fighter_2'] = "Kevin Rosier"

# full_matches_df.loc[57]['Fighter_1'] = "Anthony Macias"
# full_matches_df.loc[57]['Fighter_2'] = "He-Man Gipson"

# full_matches_df.loc[58]['Fighter_1'] = "Joel Sutton"
# full_matches_df.loc[58]['Fighter_2'] = "Jack McGlaughlin"

# full_matches_df.loc[67]['Fighter_1'] = "Onassis Parungao"
# full_matches_df.loc[67]['Fighter_2'] = "Francesco Maturi"

# full_matches_df.loc[68]['Fighter_1'] = "Joel Sutton"
# full_matches_df.loc[68]['Fighter_2'] = "Geza Kalman"

# full_matches_df.loc[76]['Fighter_1'] = "Mark Hall"
# full_matches_df.loc[76]['Fighter_2'] = "Trent Jenkins"

# full_matches_df.loc[77]['Fighter_1'] = "Joe Charles"
# full_matches_df.loc[77]['Fighter_2'] = "Scott Bessac"

# full_matches_df.loc[79]['Fighter_1'] = "Sam Adkins"
# full_matches_df.loc[79]['Fighter_2'] = "Keith Mielke"

# full_matches_df.loc[96]['Fighter_1'] = "Sam Adkins"
# full_matches_df.loc[96]['Fighter_2'] = "Felix Lee Mitchell"
    
# full_matches_df.loc[107]['Fighter_1'] = "Roberto Traven"
# full_matches_df.loc[107]['Fighter_2'] = "Dave Berry"

# full_matches_df.loc[110]['Fighter_1'] = "Scott Ferrozzo"
# full_matches_df.loc[110]['Fighter_2'] = "Sam Fulton"

# full_matches_df.loc[116]['Fighter_1'] = "Tai Bowden"
# full_matches_df.loc[116]['Fighter_2'] = "Jack Nilson"

# full_matches_df.loc[119]['Fighter_1'] = "Steve Nelmark"
# full_matches_df.loc[119]['Fighter_2'] = "Marcus Bossett"

# full_matches_df.loc[120]['Fighter_1'] = "Mark Hall"
# full_matches_df.loc[120]['Fighter_2'] = "Felix Lee Mitchell"

# full_matches_df.loc[128]['Fighter_1'] = "Justin Martin"
# full_matches_df.loc[128]['Fighter_2'] = "Eric Martin"

# full_matches_df.loc[129]['Fighter_1'] = "Nick Sanzo"
# full_matches_df.loc[129]['Fighter_2'] = "Jackie Lee"

# full_matches_df.loc[163]['Fighter_1'] = "Laverne Clark"
# full_matches_df.loc[163]['Fighter_2'] = "Josh Stuart"

# full_matches_df.loc[164]['Fighter_1'] = "Chris Brennan"
# full_matches_df.loc[164]['Fighter_2'] = "Courtney Turner"

# full_matches_df.loc[176]['Fighter_1'] = "Andre Roberts"
# full_matches_df.loc[176]['Fighter_2'] = "Harry Moskowitz"

# full_matches_df.loc[180]['Fighter_1'] = "Cesar Marscucci"
# full_matches_df.loc[180]['Fighter_2'] = "Paulo Santos"

In [ ]:
# full_matches_df.fillna("--", inplace=True)

In [25]:
full_matches_df = full_matches_df.apply(lambda x: x.str.strip() if x.dtype == "object" else x)

In [26]:
full_matches_df.to_csv("data_ufcstats/ufc-stats-matches-detailed.csv", index=False)

In [27]:
full_matches_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5607 entries, 0 to 5606
Data columns (total 39 columns):
BODY_1         5586 non-null object
BODY_2         5586 non-null object
CLINCH_1       5586 non-null object
CLINCH_2       5586 non-null object
DISTANCE_1     5586 non-null object
DISTANCE_2     5586 non-null object
Date           5607 non-null object
Event          5607 non-null object
Fighter_1      5607 non-null object
Fighter_2      5607 non-null object
GROUND_1       5586 non-null object
GROUND_2       5586 non-null object
HEAD_1         5586 non-null object
HEAD_2         5586 non-null object
KD_1           5586 non-null object
KD_2           5586 non-null object
LEG_1          5586 non-null object
LEG_2          5586 non-null object
METHOD         5607 non-null object
PASS_1         5586 non-null object
PASS_2         5586 non-null object
REFEREE        5607 non-null object
REV_1          5586 non-null object
REV_2          5586 non-null object
ROUND          5607 non-null 

# For adding new event and match data

In [ ]:
event_date = "May 30, 2020"
event_name = "UFC Fight Night: Woodley vs. Burns"
event_location = "Las Vegas, Nevada, USA"
event_url = 'http://ufcstats.com/event-details/14b9e0f2679a2205'

In [ ]:
# ufc-stats-events-list
single_df = pd.DataFrame(np.array([[event_date, event_name, event_location, event_url]]), 
                         columns=['Date', 'Event', 'Location', 'url'])
single_df.to_csv("data_ufcstats/ufc-stats-events-list.csv", mode='a', index = False, header=False)

In [ ]:
url_request = requests.get(event_url).text
soup = BeautifulSoup(url_request, 'html.parser')
rows = soup.find_all('tr',{'class':'b-fight-details__table-row'})
matches_data = []
for row in rows[1:]:
    matches_data.append([t.text.strip() for t in row.find_all('td')]) 
matches_df = pd.DataFrame(matches_data, columns=['Results', 'Fighters', 'STR', 'TD', 'SUB', 'PASS', 'WEIGHT CLASS', 'METHOD', 'ROUND', 'TIME'])

matches_df['Results_1'] = matches_df['Results'].str.split("\n",expand=True)[0]
try:
    matches_df['Results_2'] = matches_df['Results'].str.split("\n",expand=True)[3]
except:
    matches_df['Results_2'] = ""
matches_df['Fighter_1'] = matches_df['Fighters'].str.split("\n",expand=True)[0]
matches_df['Fighter_2'] = matches_df['Fighters'].str.split("\n",expand=True)[5]
matches_df['STR_1'] = matches_df['STR'].str.split("\n",expand=True)[0]
matches_df['STR_2'] = matches_df['STR'].str.split("\n",expand=True)[6]
matches_df['TD_1'] = matches_df['TD'].str.split("\n",expand=True)[0]
matches_df['TD_2'] = matches_df['TD'].str.split("\n",expand=True)[4]
matches_df['SUB_1'] = matches_df['SUB'].str.split("\n",expand=True)[0]
matches_df['SUB_2'] = matches_df['SUB'].str.split("\n",expand=True)[4]
matches_df['PASS_1'] = matches_df['PASS'].str.split("\n",expand=True)[0]
matches_df['PASS_2'] = matches_df['PASS'].str.split("\n",expand=True)[4]
matches_df['METHOD_result'] = matches_df['METHOD'].str.split("\n",expand=True)[0]
matches_df['METHOD_note'] = matches_df['METHOD'].str.split("\n",expand=True)[4]

matches_df.insert(0, "Date", event_date) 
matches_df.insert(1, "Event", event_name)
matches_df['Date']= pd.to_datetime(event_date,format= '%B %d, %Y')
matches_df = matches_df[['Date', 'Event', 'ROUND', 'TIME', 'Results_1', 'Results_2',
       'Fighter_1', 'Fighter_2', 'WEIGHT CLASS', 'STR_1', 'STR_2', 'TD_1', 'TD_2', 'SUB_1',
       'SUB_2', 'PASS_1', 'PASS_2', 'METHOD_result', 'METHOD_note']]

matches_df = matches_df.apply(lambda x: x.str.strip() if x.dtype == "object" else x)

In [ ]:
# ufc-stats-matches
matches_df.to_csv("data_ufcstats/ufc-stats-matches-all.csv", mode='a', index = False, header=False)
matches_df.head()

In [ ]:
matches_data_df = pd.DataFrame()
try:
    url_request = requests.get(event_url).text
    soup = BeautifulSoup(url_request, 'html.parser')
    table_rows = soup.find_all('tr',{'class':'js-fight-details-click'})
    data = []
    for row in table_rows:
        data.append(row['data-link'])
    df = pd.DataFrame(data, columns=['match_url'])
    df.insert(0, "Date", event_date) 
    df.insert(1, "Event", event_name)
except:
    print("Error")
matches_data_df = matches_data_df.append(df, ignore_index=True)

In [ ]:
# ufc-stats-matches-links
matches_data_df.to_csv("data_ufcstats/ufc-stats-matches-links.csv", mode='a', index = False, header=False)
matches_data_df

In [ ]:
full_matches_df = pd.DataFrame()
for index, row in matches_data_df.iterrows():
    event_name = row['Event']
    match_url = row['match_url']
    event_date = row['Date']
    print(str(index) + "\t" + event_date + "\t" + event_name)
    print(str(match_url))
    try:
        url_request = requests.get(match_url).text
        soup = BeautifulSoup(url_request, 'html.parser')
        
        fight_details = soup.find('div',{'class':'b-fight-details__fight'})
        description = [t.text.strip() for t in fight_details.find_all('p')][0]
        description = ' '.join(description.split())
        item_df = pd.DataFrame([description], columns=['description'])

        p = re.compile(r'\S+: ')
        item_df['METHOD'] = p.split(description)[1]
        item_df['ROUND'] = p.split(description)[2].strip()
        item_df['TIME'] = p.split(description)[3].split(" ")[0]
        item_df['TIME_FORMAT'] = p.split(description)[4].strip()
        if len(p.split(description)) < 6:
            item_df['REFEREE'] = ""
        else:
            item_df['REFEREE'] = p.split(description)[5]
        item_df = item_df.drop(columns=['description'])

        item_df.insert(0, "Date", event_date) 
        item_df.insert(1, "Event", event_name)
        
        table_rows = soup.find_all('table',{'cellpadding':None})

        lines = []
        for row in table_rows[0::2]:
            lines.append([t.text.strip() for t in row.find_all('td')])

        # FIGHTER	KD	SIG. STR.	SIG. STR. %	TOTAL STR.	TD	TD %	SUB. ATT	PASS	REV.
        df = pd.DataFrame(lines[0]).transpose()

        item_df['Fighter_1'] = df[0].str.split("\n",expand=True)[0]
        item_df['Fighter_2'] = df[0].str.split("\n",expand=True)[3]
        item_df['KD_1'] = df[1].str.split("\n",expand=True)[0]
        item_df['KD_2'] = df[1].str.split("\n",expand=True)[3]
        item_df['SIG_STR_1'] = df[2].str.split("\n",expand=True)[0]
        item_df['SIG_STR_2'] = df[2].str.split("\n",expand=True)[3]
        item_df['pSIG_STR_1'] = df[3].str.split("\n",expand=True)[0]
        item_df['pSIG_STR_2'] = df[3].str.split("\n",expand=True)[3]
        item_df['TOTAL_STR_1'] = df[4].str.split("\n",expand=True)[0]
        item_df['TOTAL_STR_2'] = df[4].str.split("\n",expand=True)[3]
        item_df['TD_1'] = df[5].str.split("\n",expand=True)[0]
        item_df['TD_2'] = df[5].str.split("\n",expand=True)[3]
        item_df['pTD_1'] = df[6].str.split("\n",expand=True)[0]
        item_df['pTD_2'] = df[6].str.split("\n",expand=True)[3]
        item_df['SUB_ATT_1'] = df[7].str.split("\n",expand=True)[0]
        item_df['SUB_ATT_2'] = df[7].str.split("\n",expand=True)[3]
        item_df['PASS_1'] = df[8].str.split("\n",expand=True)[0]
        item_df['PASS_2'] = df[8].str.split("\n",expand=True)[3]
        item_df['REV_1'] = df[9].str.split("\n",expand=True)[0]
        item_df['REV_2'] = df[9].str.split("\n",expand=True)[3]

        # FIGHTER	SIG. STR	SIG. STR. %	HEAD	BODY	LEG	DISTANCE	CLINCH	GROUND
        df = pd.DataFrame(lines[1]).transpose()

        item_df['HEAD_1'] = df[3].str.split("\n",expand=True)[0]
        item_df['HEAD_2'] = df[3].str.split("\n",expand=True)[3]
        item_df['BODY_1'] = df[4].str.split("\n",expand=True)[0]
        item_df['BODY_2'] = df[4].str.split("\n",expand=True)[3]
        item_df['LEG_1'] = df[5].str.split("\n",expand=True)[0]
        item_df['LEG_2'] = df[5].str.split("\n",expand=True)[3]
        item_df['DISTANCE_1'] = df[6].str.split("\n",expand=True)[0]
        item_df['DISTANCE_2'] = df[6].str.split("\n",expand=True)[3]
        item_df['CLINCH_1'] = df[7].str.split("\n",expand=True)[0]
        item_df['CLINCH_2'] = df[7].str.split("\n",expand=True)[3]
        item_df['GROUND_1'] = df[8].str.split("\n",expand=True)[0]
        item_df['GROUND_2'] = df[8].str.split("\n",expand=True)[3]
        
    except:
        print("Error")
        item_df['Fighter_1'] = match_url
    full_matches_df = full_matches_df.append(item_df, ignore_index=True)

In [ ]:
full_matches_df = full_matches_df.apply(lambda x: x.str.strip() if x.dtype == "object" else x)
full_matches_df.sort_index(axis=1, inplace=True)

# ufc-stats-matches-detailed
full_matches_df.to_csv("data_ufcstats/ufc-stats-matches-detailed.csv", mode='a', index = False, header=False)

full_matches_df

In [ ]:
url_request = requests.get("http://ufcstats.com/fight-details/27bfb9205119f23b").text
soup = BeautifulSoup(url_request, 'html.parser')

fight_details = soup.find('div',{'class':'b-fight-details__fight'})
description = [t.text.strip() for t in fight_details.find_all('p')][0]
description = ' '.join(description.split())
item_df = pd.DataFrame([description], columns=['description'])

p = re.compile(r'\S+: ')
item_df['METHOD'] = p.split(description)[1]
item_df['ROUND'] = p.split(description)[2].strip()
item_df['TIME'] = p.split(description)[3].split(" ")[0]
item_df['TIME_FORMAT'] = p.split(description)[4].strip()
if len(p.split(description)) < 6:
    item_df['REFEREE'] = ""
else:
    item_df['REFEREE'] = p.split(description)[5]
# item_df = item_df.drop(columns=['description'])

# item_df.insert(0, "Date", event_date) 
# item_df.insert(1, "Event", event_name)

table_rows = soup.find_all('table',{'cellpadding':None})

lines = []
for row in table_rows[0::2]:
    lines.append([t.text.strip() for t in row.find_all('td')])

# FIGHTER	KD	SIG. STR.	SIG. STR. %	TOTAL STR.	TD	TD %	SUB. ATT	PASS	REV.
df = pd.DataFrame(lines[0]).transpose()

item_df['Fighter_1'] = df[0].str.split("\n",expand=True)[0]
item_df['Fighter_2'] = df[0].str.split("\n",expand=True)[3]
item_df['KD_1'] = df[1].str.split("\n",expand=True)[0]
item_df['KD_2'] = df[1].str.split("\n",expand=True)[3]
item_df['SIG_STR_1'] = df[2].str.split("\n",expand=True)[0]
item_df['SIG_STR_2'] = df[2].str.split("\n",expand=True)[3]
item_df['pSIG_STR_1'] = df[3].str.split("\n",expand=True)[0]
item_df['pSIG_STR_2'] = df[3].str.split("\n",expand=True)[3]
item_df['TOTAL_STR_1'] = df[4].str.split("\n",expand=True)[0]
item_df['TOTAL_STR_2'] = df[4].str.split("\n",expand=True)[3]
item_df['TD_1'] = df[5].str.split("\n",expand=True)[0]
item_df['TD_2'] = df[5].str.split("\n",expand=True)[3]
item_df['pTD_1'] = df[6].str.split("\n",expand=True)[0]
item_df['pTD_2'] = df[6].str.split("\n",expand=True)[3]
item_df['SUB_ATT_1'] = df[7].str.split("\n",expand=True)[0]
item_df['SUB_ATT_2'] = df[7].str.split("\n",expand=True)[3]
item_df['PASS_1'] = df[8].str.split("\n",expand=True)[0]
item_df['PASS_2'] = df[8].str.split("\n",expand=True)[3]
item_df['REV_1'] = df[9].str.split("\n",expand=True)[0]
item_df['REV_2'] = df[9].str.split("\n",expand=True)[3]

# FIGHTER	SIG. STR	SIG. STR. %	HEAD	BODY	LEG	DISTANCE	CLINCH	GROUND
df = pd.DataFrame(lines[1]).transpose()

item_df['HEAD_1'] = df[3].str.split("\n",expand=True)[0]
item_df['HEAD_2'] = df[3].str.split("\n",expand=True)[3]
item_df['BODY_1'] = df[4].str.split("\n",expand=True)[0]
item_df['BODY_2'] = df[4].str.split("\n",expand=True)[3]
item_df['LEG_1'] = df[5].str.split("\n",expand=True)[0]
item_df['LEG_2'] = df[5].str.split("\n",expand=True)[3]
item_df['DISTANCE_1'] = df[6].str.split("\n",expand=True)[0]
item_df['DISTANCE_2'] = df[6].str.split("\n",expand=True)[3]
item_df['CLINCH_1'] = df[7].str.split("\n",expand=True)[0]
item_df['CLINCH_2'] = df[7].str.split("\n",expand=True)[3]
item_df['GROUND_1'] = df[8].str.split("\n",expand=True)[0]
item_df['GROUND_2'] = df[8].str.split("\n",expand=True)[3]

In [ ]:
item_df